# Project Idea: 
* Get SEC data
* Create standardized balance sheets for a single company over time 
* Do a simple DCF Analysis 
* Turn it into modular functions
* Maybe create a dashboard(?)

Links: 
https://www.sec.gov/dera/data/financial-statement-data-sets.html
https://www.sec.gov/dera/data/financial-statement-and-notes-data-set.html


In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from bs4 import BeautifulSoup
import requests
import os
import re
import zipfile
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas_datareader as pdr
from itertools import chain
os.environ["ALPHAVANTAGE_API_KEY"] = "H6AXUXK0Z2CBTV59" 

In [2]:
# Variables we can parse out
name =  "APPLE INC"#E.g."MICROSOFT CORP"

In [2]:
ticker_cik = pd.read_table("https://www.sec.gov/include/ticker.txt", names = ["Ticker","CIK"])
ticker_cik.to_csv("Ticker_to_CIK_Mapping.csv")#Ticker to SIC Code
sic_code_to_name = pd.read_html("https://www.sec.gov/info/edgar/siccodes.htm")[0] #For Sectors
sic_code_to_name.to_csv("sic_code_to_name.csv") 

## Getting the Meta-data

In [4]:
#Downloading the data 
dtypes = {"fy":str,
          "fye":str,
          "period":str,
          "ddate":str}
num = pd.read_csv("2021q1/num.txt", sep="\t", dtype = dtypes)
sub = pd.read_csv("2021q1/sub.txt", sep="\t", dtype = dtypes)

In [5]:
symbol = ticker_cik[ticker_cik["CIK"]==sub[sub["name"]==name]["cik"].iloc[0]]["Ticker"].iloc[0] #Getting a Ticker

## Getting the Data

##### Short excursion: Calculating the Alpha of the stock
###### 1. Using Fama-French 3 Factor Model

In [208]:
#Getting stock prices and factors 
start = "2007-01-01"
end = datetime.now()

share_price_ts = pdr.DataReader(symbol, "av-weekly-adjusted", start=start,end=end)["adjusted close"]

ff_factors = pd.read_csv("Factor Data/Research_Data_Factors_weekly.csv",skiprows=4, index_col=0).dropna()
ff_factors.index = pd.DatetimeIndex(ff_factors.index)
ff_factors = ff_factors.resample("W").last()

share_price_ts.index = pd.DatetimeIndex(share_price_ts.index)
share_price_ts = share_price_ts.resample("W").last().pct_change()
ff_df = pd.concat([share_price_ts,ff_factors], axis=1)
ff_df = ff_df.dropna()

In [209]:
X = ff_df[['Mkt-RF', 'SMB', 'HML',"RF"]]
X = sm.add_constant(X)

model = sm.OLS(endog = ff_df["adjusted close"], exog = X, missing = "drop")
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         adjusted close   R-squared:                       0.410
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     128.7
Date:                Sun, 20 Jun 2021   Prob (F-statistic):           1.92e-83
Time:                        12:59:00   Log-Likelihood:                 1476.6
No. Observations:                 747   AIC:                            -2943.
Df Residuals:                     742   BIC:                            -2920.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.001      1.498      0.135      -0.001       0.005
Mkt-RF         0.0113      0.001     22.109      0.000       0.010       0.012
SMB           -0.0003      0.001     -0.341      0.733      -0.002       0.002
HML           -0.0054      0.001     -7.247      0.000      -0.007      -0.004
RF             0.0726      0.047      1.558      0.120      -0.019       0.164
==============================================================================
Omnibus:                       62.487   Durbin-Watson:                   2.055
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              317.304
Skew:                          -0.102   Prob(JB):                     1.25e-69
Kurtosis:                       6.186   Cond. No.                         105.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [210]:
information_ratio_ff = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))
information_ratio_ff

0.11086430836822345

In [211]:
start = "2007-01-01"
end = datetime.now()

share_price_ts = pdr.DataReader(symbol, "av-weekly-adjusted", start=start,end=end)["adjusted close"]
market_factor = pdr.DataReader("VTSAX", "av-weekly-adjusted", start=start,end=end)["adjusted close"]
risk_free = pdr.DataReader("SHY", "av-weekly-adjusted", start=start,end=end)["adjusted close"]

#Calculating Alpha
return_equity = share_price_ts.pct_change()
return_market = market_factor.pct_change()
return_risk_free = risk_free.pct_change()
capm_df = pd.concat([return_equity,return_market,return_risk_free], axis=1).dropna()
capm_df.columns = ["Equity Return", "Market Return", "Risk-Free Return"]
capm_df["Risk Premium"] = capm_df["Market Return"] - capm_df["Risk-Free Return"]
capm_df.index = pd.DatetimeIndex(capm_df.index)

X = capm_df[["Risk-Free Return", "Risk Premium"]]
X = sm.add_constant(X)

model = sm.OLS(endog = capm_df["Equity Return"], exog = X, missing = "drop")
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Equity Return   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     214.6
Date:                Sun, 20 Jun 2021   Prob (F-statistic):           1.98e-74
Time:                        12:59:16   Log-Likelihood:                 1463.0
No. Observations:                 753   AIC:                            -2920.
Df Residuals:                     750   BIC:                            -2906.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0043      0.001      3.293      0.001       0.002       0.007
Risk-Free Return    -0.0928      0.867     -0.107      0.915      -1.794       1.609
Risk Premium         0.9605      0.051     18.675      0.000       0.859       1.061
==============================================================================
Omnibus:                       60.083   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              289.728
Skew:                          -0.100   Prob(JB):                     1.22e-63
Kurtosis:                       6.032   Cond. No.                         685.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [212]:
information_ratio_capm = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))
information_ratio_capm

0.21349946731878344

## Back to topic: Getting Company reports

In [87]:
#Selecting a company and showing reports for the quarter
entity_sub = sub[sub["name"]==name]
entity_sub[["form","fy","fp","fye","period","adsh","accepted"]]


,form,fy,fp,fye,period,adsh,accepted
267,8-K,NaN,NaN,0930,20201231,0001193125-21-001982,2021-01-05 16:32:00.0
2551,10-Q,2021,Q1,0930,20201231,0000320193-21-000010,2021-01-27 18:03:00.0
3037,8-K,NaN,NaN,0930,20210131,0000320193-21-000009,2021-01-27 16:31:00.0
5356,8-K,NaN,NaN,0930,20210131,0001193125-21-032394,2021-02-08 16:31:00.0
9015,8-K,NaN,NaN,0930,20210228,0001193125-21-054710,2021-02-24 16:31:00.0


In [7]:
#filtering relevant data
quaterly_report_adsh = entity_sub[entity_sub["form"]=="10-Q"]["adsh"].iloc[0]
quaterly_report_period = entity_sub[entity_sub["form"]=="10-Q"]["period"].iloc[0]
entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh]
entity_num_quarterly_period = entity_num_quarterly[entity_num_quarterly["ddate"]==quaterly_report_period]
eps_t = entity_num_quarterly_period[entity_num_quarterly_period["tag"]=="EarningsPerShareBasic"]["value"].iloc[0]

In [8]:
#Show current EPS
eps_t

1.7

## Checking the process: Does this work for 2010 data as well...?

In [129]:
dtypes = {"fy":str,
          "fye":str,
          "period":str,
          "ddate":str}
num = pd.read_csv("2010q1/num.txt", sep="\t", dtype = dtypes)
sub = pd.read_csv("2010q1/sub.txt", sep="\t", dtype = dtypes)



In [130]:
entity_sub = sub[sub["name"]==name]
entity_sub[["form","fy","fp","fye","period","adsh"]]

,form,fy,fp,fye,period,adsh
482,10-Q,2010,Q1,0930,20091231,0001193125-10-012085
483,10-K/A,2009,FY,0930,20090930,0001193125-10-012091


In [131]:
quaterly_report_adsh = entity_sub[entity_sub["form"]=="10-Q"]["adsh"].iloc[0]
quaterly_report_period = entity_sub[entity_sub["form"]=="10-Q"]["period"].iloc[0]
entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh]
entity_num_quarterly_period = entity_num_quarterly[entity_num_quarterly["ddate"]==quaterly_report_period]

In [132]:
eps_t = entity_num_quarterly_period[entity_num_quarterly_period["tag"]=="EarningsPerShareBasic"]["value"].iloc[0]
eps_t

3.74

## It does!

## Setting up a process to iterate through files

In [11]:
#Unzip all files in the database
database = os.listdir("Database")
for file in database: 
    if "zip" in file: 
        with zipfile.ZipFile("Database/"+file,"r") as zip_ref:
            zip_ref.extractall("Database/"+file.replace("zip",""))
        os.remove("Database/"+file)

In [14]:
#loading database again without zips
database = os.listdir("Database")
eps_list = list()
date_list = list()
for file in database[0:12]:
    dtypes = {"fy":str,
              "fye":str,
              "period":str,
              "ddate":str}
    num = pd.read_csv("Database/"+file+"/num.txt", sep="\t", dtype = dtypes)
    sub = pd.read_csv("Database/"+file+"/sub.txt", sep="\t", dtype = dtypes)
    report_type = "10-Q"
    
    if name not in sub["name"].unique(): #If the firm we are looking for is not in the metadata for that quarter, we dont need to get into numbers
        print(name+" not in "+file)
    else: 
        entity_sub = sub[sub["name"]==name]
        print("Reports for "+file)
        print(entity_sub[["form","fy","fp","fye","period","adsh"]])
        if "10-Q" not in entity_sub["form"].unique(): #If our firm did not release a 10-Q, we check if we find the 10-K (they do not release both in the quarter after their year end)
            print("No 10-Q in "+file+", using 10-K instead")
            report_type = "10-K"

        quaterly_report_adsh = entity_sub[entity_sub["form"]==report_type]["adsh"].iloc[0] #fetch report identifier "adsh"
        quaterly_report_period = entity_sub[entity_sub["form"]==report_type]["period"].iloc[0] #fetch report accounting period
        entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh] #fetch report
        entity_num_quarterly_period = entity_num_quarterly[entity_num_quarterly["ddate"]==quaterly_report_period] #Only get the data for the current period
        eps_t = entity_num_quarterly_period[entity_num_quarterly_period["tag"]=="EarningsPerShareBasic"]["value"].iloc[0]

        date_list.append(quaterly_report_period)
        eps_list.append(eps_t)

        print("EPS in "+str(quaterly_report_period)+": "+str(eps_t))

APPLE INC not in 2009q1
APPLE INC not in 2009q2
Reports for 2009q3
   form    fy  fp   fye    period                  adsh
5  10-Q  2009  Q3  0930  20090630  0001193125-09-153165
EPS in 20090630: 1.38


c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Reports for 2009q4
    form    fy  fp   fye    period                  adsh
67  10-K  2009  FY  0930  20090930  0001193125-09-214859
No 10-Q in 2009q4, using 10-K instead
EPS in 20090930: 6.39
Reports for 2010q1
       form    fy  fp   fye    period                  adsh
482    10-Q  2010  Q1  0930  20091231  0001193125-10-012085
483  10-K/A  2009  FY  0930  20090930  0001193125-10-012091
EPS in 20091231: 3.74
Reports for 2010q2
    form    fy  fp   fye    period                  adsh
16  10-Q  2010  Q2  0930  20100331  0001193125-10-088957
EPS in 20100331: 7.12
Reports for 2010q3
    form    fy  fp   fye    period                  adsh
27  10-Q  2010  Q3  0930  20100630  0001193125-10-162840
EPS in 20100630: 10.69
Reports for 2010q4
     form    fy  fp   fye    period                  adsh
167  10-K  2010  FY  0930  20100930  0001193125-10-238044
No 10-Q in 2010q4, using 10-K instead
EPS in 20100930: 4.71
Reports for 2011q1
      form    fy  fp   fye    period                  adsh
14

In [15]:
#Construct EPS Data Frame
df = pd.DataFrame({"Date":date_list,"EPS":eps_list})
df.index = pd.DatetimeIndex(df["Date"])
df = df.drop("Date", axis=1)
df

,EPS
Date,
2009-06-30,1.38
2009-09-30,6.39
2009-12-31,3.74
2010-03-31,7.12
2010-06-30,10.69
2010-09-30,4.71
2010-12-31,6.53
2011-03-31,6.49
2011-06-30,20.91


In [8]:
dtypes = {"fy":str,
          "fye":str,
          "period":str,
          "ddate":str}
num = pd.read_csv("Database/2010q4/num.txt", sep="\t", dtype = dtypes)
sub = pd.read_csv("Database/2010q4/sub.txt", sep="\t", dtype = dtypes)
entity_sub = sub[sub["name"]==name]
entity_sub[["form","fy","fp","fye","period","adsh"]]


,form,fy,fp,fye,period,adsh
167,10-K,2010,FY,0930,20100930,0001193125-10-238044


In [260]:
quaterly_report_adsh = entity_sub[entity_sub["form"]=="10-K"]["adsh"].iloc[0]
quaterly_report_period = entity_sub[entity_sub["form"]=="10-K"]["period"].iloc[0]
entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh]
entity_num_quarterly_period = entity_num_quarterly[entity_num_quarterly["ddate"]==quaterly_report_period]

In [262]:
eps_t = entity_num_quarterly_period[entity_num_quarterly_period["tag"]=="EarningsPerShareBasic"]["value"]#.iloc[0]
eps_t

1986879    3.31
1986886    0.74
Name: value, dtype: float64

## Yep, this seems to work as well. Let's make a process to get *all* the data, not just one variable

###### Get all firms and all sector (SIC) codes

In [3]:
all_firms_list = list()
database = os.listdir("Database")
for file in database:
    dtypes = {"fy":str,
              "fye":str,
              "period":str,
              "ddate":str,
             "sic":str}
#    num = pd.read_csv("Database/"+file+"/num.txt", sep="\t", dtype = dtypes)
    sub = pd.read_csv("Database/"+file+"/sub.txt", sep="\t", dtype = dtypes)
    sub["name_sic_cik"] = sub["name"]+"_"+sub["sic"].astype(str)+"_"+sub["cik"].astype(str)
    all_firms_list.append(sub["name_sic_cik"].unique())
all_firms_list = set(list(chain(*all_firms_list)))

firms_list = list()
for i in range(0,len(all_firms_list)):
    firms_dic = {"Firm":None,
                 "SIC":None,
                 "CIK":None}
    firms_dic["Firm"] = (list(all_firms_list)[i].split("_"))[0]
    firms_dic["SIC"] = (list(all_firms_list)[i].split("_"))[1]
    firms_dic["CIK"] = (list(all_firms_list)[i].split("_"))[2]
    firms_list.append(firms_dic)

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
firms_dict_df = pd.DataFrame.from_dict(firms_list)
firms_dict_df.head()

,Firm,SIC,CIK
0,INDEPENDENT FILM DEVELOPMENT CORP,6799,1425883
1,MARATHON PETROLEUM CORP,2911,1510295
2,PROVIDENT FINANCIAL SERVICES INC,6035,1178970
3,MYOS CORP,2834,1402479
4,GARNER INVESTMENTS INC,1381,1087734


###### Keep only firms that have a market ticker:

In [5]:
ticker_cik["CIK"] = ticker_cik["CIK"].astype(str)
firms_df = pd.merge(firms_dict_df,ticker_cik, left_on="CIK", right_on="CIK").drop_duplicates(["SIC","CIK","Ticker"])
firms_df

,Firm,SIC,CIK,Ticker
0,MARATHON PETROLEUM CORP,2911,1510295,mpc
1,PROVIDENT FINANCIAL SERVICES INC,6035,1178970,pfs
2,MYOS CORP,2834,1402479,mdvl
3,ATLAS THERAPEUTICS CORP,5900,1402479,mdvl
5,MYOS CORP,2020,1402479,mdvl
...,...,...,...,...
12749,KITE REALTY GROUP TRUST,6798,1286043,krg
12750,WILLIS LEASE FINANCE CORP,5080,1018164,wlfc
12751,"RING ENERGY, INC.",1311,1384195,rei
12752,TJX COMPANIES INC /DE/,5651,109198,tjx


###### FInd all firms in a specific industry

In [6]:
sector = "ELECTRONIC COMPUTERS"
sic_sector = str(sic_code_to_name[sic_code_to_name["Industry Title"] == sector]["SIC Code"].iloc[0])
sic_code_to_name[sic_code_to_name["Industry Title"] == sector]

,SIC Code,Office,Industry Title
169,3571,Office of Technology,ELECTRONIC COMPUTERS


In [7]:
firms_df[firms_df["SIC"]==sic_sector]

,Firm,SIC,CIK,Ticker
1807,DELL TECHNOLOGIES INC.,3571,1571996,dell
5232,HUAMI CORP,3571,1720446,zepp
6944,"SECTOR 5, INC.",3571,1550737,sfiv
7047,"OMNICELL, INC",3571,926326,omcl
7086,NICE LTD.,3571,1003935,nice
7087,NICE LTD.,3571,1003935,ncsyf
7135,CONCURRENT COMPUTER CORP/DE,3571,749038,ccur
7136,CONCURRENT COMPUTER CORP/DE,3571,749038,ccurd
9901,TERADATA CORP /DE/,3571,816761,tdc
10636,"SUPER MICRO COMPUTER, INC.",3571,1375365,smci


##### Run Process for a single firm

In [43]:
t = time.process_time()
start = "2007-01-01"
end = datetime.now()
market_factor = pdr.DataReader("VTSAX", "av-weekly-adjusted", start=start,end=end)["adjusted close"]
risk_free = pdr.DataReader("SHY", "av-weekly-adjusted", start=start,end=end)["adjusted close"]
return_market = market_factor.pct_change()
return_risk_free = risk_free.pct_change()
elapsed_time = time.process_time() - t
elapsed_time

0.0625

In [49]:
numdict1 = {}
for x,file in enumerate(database): 
    numdict1[file] = num_list[x]   
t = time.process_time()
#loading database again without zips
database = os.listdir("Database")
report_list = list() #this will get all reports
date_list = list()
annual_report_list = list() #from this we can make a balance sheet file 
annual_report_date_list = list() #dates for balance sheet file
market_data_list = list() #market_data list for regressions etc. 
share_price_ts = []

CIK = "320193"
name = str(firms_df[firms_df["CIK"]==CIK]["Firm"].iloc[0])
print("IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII")
print("Starting with: "+name)
for file in database:
    dtypes = {"fy":str,
              "fye":str,
              "period":str,
              "ddate":str,
              "sic":str,
             "cik":str}

    sub = pd.read_csv("Database/"+file+"/sub.txt", sep="\t", dtype = dtypes)
    report_type = "10-Q"
    
    if CIK not in sub["cik"].unique(): #If the firm we are looking for is not in the metadata for that quarter, we dont need to get into numbers
        print(name+" not in "+file)
        continue 
   
    entity_sub = sub[sub["cik"]==CIK]
    print("Report that was published in: "+file)
    print(entity_sub[["form","fy","fp","fye","period","adsh","accepted"]])
    if "10-Q" not in entity_sub["form"].unique(): #If our firm did not release a 10-Q, we check if we find the 10-K (they do not release both in the quarter after their year end)
        print("No 10-Q in "+file+", fetching 10-K instead")
        report_type = "10-K"
        if "10-K" not in entity_sub["form"].unique(): #If our firm released neither 10-Q nor a 10-K 
            print("No 10-Q or 10-K for "+file+" skipping to next quarter")
            continue
    
    num = numdict1[file]
    #Get the meta data
    quaterly_report_adsh = entity_sub[entity_sub["form"]==report_type]["adsh"].iloc[0] #fetch report identifier "adsh"
    quaterly_report_period = entity_sub[entity_sub["form"]==report_type]["period"].iloc[0] #fetch report accounting period
    
    #Get the market data
    first_filing_date = datetime.strptime(entity_sub[entity_sub["form"]==report_type]["accepted"].iloc[0].replace(".0",""), "%Y-%m-%d %H:%M:%S") #fetch report acceptance date
        
    print("Fetching Report that was published in :  "+str(quaterly_report_period))
    
    #For the first date for which we have accounting data, download all the data we need for the CAPM 
    if ((date_list==[]) and (type(first_filing_date)==datetime)): #only do this once
        #Get the symbol for stock market API
        #Setting up dictionary
        regression_stats_dict = {"filing_date":None,
                                "next_year_beta":None,
                                "next_year_beta_tstat":None,
                                "next_year_alpha":None,
                                "next_year_alpha_tstat":None,
                                "next_year_IR":None,
                                "next_year_market_return":None,
                                "next_year_equity_return":None,
                                "last_year_start_date":None,
                                "last_year_beta":None, 
                                "last_year_beta_tstat":None,
                                "last_year_alpha":None, 
                                "last_year_alpha_tstat":None,
                                "last_year_IR": None,
                                "last_year_market_return":None,
                                "last_year_equity_return":None,
                                "last_year_vola":None, 
                                "last_year_vola_explained_by_market":None}    
        
        
        #Getting stock prices and factors 
        start = first_filing_date-pd.Timedelta(52,"W")
        end = datetime.now()

        try:
            share_price_ts = pdr.DataReader(str(firms_df[firms_df["CIK"]==CIK]["Ticker"].iloc[0]), "av-weekly-adjusted", start=start,end=end)["adjusted close"]
        except ValueError:
            try:
                share_price_ts = pdr.DataReader(str(firms_df[firms_df["CIK"]==CIK]["Ticker"].iloc[1]), "av-weekly-adjusted", start=start,end=end)["adjusted close"]
            except ValueError:
                print('No Stock Prices available')
                #Get the accounting data 
                entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh] #fetch report
                entity_num_quarterly_period = entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                                   entity_num_quarterly["qtrs"].isin([0,1])] #Only get the data for the last quarter
                date_list.append(quaterly_report_period)
                report_list.append(entity_num_quarterly_period)

                if report_type == "10-K":
                    annual_report_list.append(entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                                    entity_num_quarterly["qtrs"].isin([0,4])])#Get last year data
                    annual_report_date_list.append(quaterly_report_period)
                    print("Also saving this as 10-K report")
                print("################################################## Next Period#################################################")
                continue

        return_equity = share_price_ts.pct_change()
        capm_df = pd.concat([return_equity,return_market,return_risk_free], axis=1).dropna()
        capm_df.columns = ["Equity Return", "Market Return", "Risk-Free Return"]
        capm_df["Risk_Premium_Beta"] = capm_df["Market Return"] - capm_df["Risk-Free Return"]         
        capm_df.index = pd.DatetimeIndex(capm_df.index)
            
        
    #Next year Stock analysis:
    if type(first_filing_date) == datetime:
        #Setup and run regression
        period_start = first_filing_date
        period_end = first_filing_date+pd.Timedelta(52,"W")
        reg_df = capm_df.loc[period_start:period_end]
        X = reg_df[["Risk-Free Return", "Risk_Premium_Beta"]]
        X = sm.add_constant(X)
        model = sm.OLS(endog = reg_df["Equity Return"], exog = X, missing = "drop")
        results = model.fit()
        information_ratio_capm = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))
       
        #Printing relevant data
        print("Equity Market Stats for 1Y since Filing on: "+str(first_filing_date))
        print(results.params)
        print("With T-Values:")
        print(results.tvalues)
        print("Information Ratio last year: "+str(information_ratio_capm))
        
        #Saving Data
        regression_stats_dict["filing_date"] = first_filing_date
        regression_stats_dict["next_year_beta"] = results.params["Risk_Premium_Beta"]
        regression_stats_dict["next_year_beta_tstat"] = results.tvalues["Risk_Premium_Beta"]
        regression_stats_dict["next_year_alpha"] = results.params["const"]
        regression_stats_dict["next_year_alpha_tstat"] = results.tvalues["const"]
        regression_stats_dict["next_year_IR"] = information_ratio_capm
        regression_stats_dict["next_year_market_return"] = (reg_df["Market Return"]+1).iloc[-1]-1
        regression_stats_dict["next_year_equity_return"] = (reg_df["Equity Return"]+1).iloc[-1]-1
        
        
        #Make dataframe and store it to list here already if we have not enough data for historic analysis
        if len(capm_df.loc[first_filing_date-pd.Timedelta(52,"w"):first_filing_date])< 26:
            market_data_df = pd.DataFrame.from_dict(regression_stats_dict,orient='index')
            market_data_df.columns = [quaterly_report_period]
            market_data_list.append(market_data_df)
            
    #Historic Stock Return Analysis if we have 26 weeks of data (1/2 year)
    if len(capm_df.loc[first_filing_date-pd.Timedelta(52,"w"):first_filing_date])>= 26:
        #Setup and run regression
        period_start = first_filing_date-pd.Timedelta(52,"w")
        period_end = first_filing_date
        reg_df = capm_df.loc[period_start:period_end]
        X = reg_df[["Risk-Free Return", "Risk_Premium_Beta"]]
        X = sm.add_constant(X)
        model = sm.OLS(endog = reg_df["Equity Return"], exog = X, missing = "drop")
        results = model.fit()
        information_ratio_capm = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))
        
        #Printing relevant data
        print("1Y Historic Equity Market Stats (filing between): "+str(period_start)+" and "+str(period_end))
        print(results.params)
        print("With T-Values:")
        print(results.tvalues)
        print("Information Ratio next year:"+str(information_ratio_capm))
        
        #saving relevant data
        regression_stats_dict["last_year_start_date"] = reg_df.first_valid_index()
        regression_stats_dict["last_year_beta"] = results.params["Risk_Premium_Beta"]
        regression_stats_dict["last_year_beta_tstat"]= results.tvalues["Risk_Premium_Beta"]
        regression_stats_dict["last_year_alpha"]=results.params["const"]
        regression_stats_dict["last_year_alpha_tstat"]=results.tvalues["const"]
        regression_stats_dict["last_year_IR"] = information_ratio_capm
        regression_stats_dict["last_year_market_return"]=(reg_df["Market Return"]+1).iloc[-1]-1
        regression_stats_dict["last_year_equity_return"]=(reg_df["Equity Return"]+1).iloc[-1]-1
        regression_stats_dict["last_year_vola"]= reg_df["Equity Return"].std()*np.sqrt(len(reg_df))
        
        #Explaining Vola of Stock with Market: 
        X = reg_df["Equity Return"]
        X = sm.add_constant(X)
        model = sm.OLS(endog = reg_df["Risk_Premium_Beta"], exog = X , missing = "drop")
        results = model.fit()
        regression_stats_dict["last_year_vola_explained_by_market"]=results.params["Equity Return"]
        
        #Make dataframe and store it to list
        market_data_df = pd.DataFrame.from_dict(regression_stats_dict,orient='index')
        market_data_df.columns = [quaterly_report_period]
        market_data_list.append(market_data_df)

    #Get the accounting data 
    entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh] #fetch report
    entity_num_quarterly_period = entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                       entity_num_quarterly["qtrs"].isin([0,1])] #Only get the data for the last quarter
    date_list.append(quaterly_report_period)
    report_list.append(entity_num_quarterly_period)



    if report_type == "10-K":
        annual_report_list.append(entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                        entity_num_quarterly["qtrs"].isin([0,4])])#Get last year data
        annual_report_date_list.append(quaterly_report_period)
        print("Also saving this as 10-K report")

    print("################################################## Next Period#################################################")


annual_report_long = pd.concat(annual_report_list)
annual_reports_complete_long = pd.pivot_table(annual_report_long, index= "ddate",columns = "tag", values="value")
if len(share_price_ts)!=0:
    market_data_results_wide = pd.concat(market_data_list,axis=1)
    market_data_results_long = market_data_results_wide.transpose()
    complete_data_long = pd.merge(annual_reports_complete_long,market_data_results_long, left_index=True, right_index=True)
    complete_data_long.index = pd.DatetimeIndex(complete_data_long.index)
    complete_data_long.index.name = str(name)+"_"+str(entity_sub["sic"].iloc[0])
    complete_data_long.transpose().to_excel("Output/"+str(name)+"_complete.xlsx")


elapsed_time = time.process_time() - t
print("Processing Time in Minutes: "+str(elapsed_time/60))
print(str(name)+" Done")
print("IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII")

del numdict1

IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
Starting with: APPLE INC
APPLE INC not in 2009q1
APPLE INC not in 2009q2
Report that was published in: 2009q3
   form    fy  fp   fye    period                  adsh               accepted
5  10-Q  2009  Q3  0930  20090630  0001193125-09-153165  2009-07-22 16:42:00.0
Fetching Report that was published in :  20090630
Equity Market Stats for 1Y since Filing on: 2009-07-22 16:42:00
const                0.007388
Risk-Free Return    -0.159513
Risk_Premium_Beta    1.106974
dtype: float64
With T-Values:
const                1.690561
Risk-Free Return    -0.063060
Risk_Premium_Beta    6.222601
dtype: float64
Information Ratio last year: 0.441961483720842
1Y Historic Equity Market Stats (filing between): 2008-07-23 16:42:00 and 2009-07-22 16:42:00
const                0.011840
Risk-Free Return    -8.865317
Risk_Premium_Beta    0.716709
dtype: float64
With T-Values:
const               

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Fetching Report that was published in :  20091231
Equity Market Stats for 1Y since Filing on: 2010-01-25 16:24:00
const                0.004141
Risk-Free Return     3.985120
Risk_Premium_Beta    1.413729
dtype: float64
With T-Values:
const                1.077473
Risk-Free Return     1.368857
Risk_Premium_Beta    8.700632
dtype: float64
Information Ratio last year: 0.2735708773801681
1Y Historic Equity Market Stats (filing between): 2009-01-26 16:24:00 and 2010-01-25 16:24:00
const                0.011198
Risk-Free Return    -1.936890
Risk_Premium_Beta    0.921582
dtype: float64
With T-Values:
const                2.627227
Risk-Free Return    -0.928669
Risk_Premium_Beta    7.247782
dtype: float64
Information Ratio next year:0.6842673012435552
################################################## Next Period#################################################
Report that was published in: 2010q2
    form    fy  fp   fye    period                  adsh  \
16  10-Q  2010  Q2  0930  20100331  0

Fetching Report that was published in :  20111231
Equity Market Stats for 1Y since Filing on: 2012-01-25 16:42:00
const                0.000867
Risk-Free Return     1.033254
Risk_Premium_Beta    1.160341
dtype: float64
With T-Values:
const                0.171462
Risk-Free Return     0.101664
Risk_Premium_Beta    3.814601
dtype: float64
Information Ratio last year: 0.04179529701367348
1Y Historic Equity Market Stats (filing between): 2011-01-26 16:42:00 and 2012-01-25 16:42:00
const                0.004878
Risk-Free Return    -0.118146
Risk_Premium_Beta    0.868647
dtype: float64
With T-Values:
const                1.025336
Risk-Free Return    -0.028920
Risk_Premium_Beta    5.490113
dtype: float64
Information Ratio next year:0.2565142493235193
################################################## Next Period#################################################
Report that was published in: 2012q2
      form    fy  fp   fye    period                  adsh  \
1403  10-Q  2012  Q2  0930  2012033

Report that was published in: 2014q1
     form    fy  fp   fye    period                  adsh  \
661  10-Q  2014  Q1  0930  20131231  0001193125-14-024487   

                  accepted  
661  2014-01-28 17:07:00.0  
Fetching Report that was published in :  20131231
Equity Market Stats for 1Y since Filing on: 2014-01-28 17:07:00
const                0.004520
Risk-Free Return     3.144908
Risk_Premium_Beta    1.119471
dtype: float64
With T-Values:
const                1.056797
Risk-Free Return     0.601223
Risk_Premium_Beta    3.298271
dtype: float64
Information Ratio last year: 0.2752797834126994
1Y Historic Equity Market Stats (filing between): 2013-01-29 17:07:00 and 2014-01-28 17:07:00
const                0.003410
Risk-Free Return    -3.476862
Risk_Premium_Beta    0.544081
dtype: float64
With T-Values:
const                0.619432
Risk-Free Return    -0.436511
Risk_Premium_Beta    1.391856
dtype: float64
Information Ratio next year:0.15523603966565033
############################

     form    fy  fp   fye    period                  adsh  \
481  10-Q  2016  Q1  0930  20151231  0001193125-16-439878   

                  accepted  
481  2016-01-27 16:32:00.0  
Fetching Report that was published in :  20151231
Equity Market Stats for 1Y since Filing on: 2016-01-27 16:32:00
const                0.000118
Risk-Free Return    -0.008076
Risk_Premium_Beta    1.014089
dtype: float64
With T-Values:
const                0.026356
Risk-Free Return    -0.002273
Risk_Premium_Beta    3.496108
dtype: float64
Information Ratio last year: 0.006483369002382382
1Y Historic Equity Market Stats (filing between): 2015-01-28 16:32:00 and 2016-01-27 16:32:00
const                0.000344
Risk-Free Return     0.937909
Risk_Premium_Beta    1.277858
dtype: float64
With T-Values:
const                0.096146
Risk-Free Return     0.304229
Risk_Premium_Beta    6.687742
dtype: float64
Information Ratio next year:0.02273317075542131
################################################## Next Period#

Fetching Report that was published in :  20171231
Equity Market Stats for 1Y since Filing on: 2018-02-02 08:01:00
const                0.000655
Risk-Free Return     3.600796
Risk_Premium_Beta    1.060082
dtype: float64
With T-Values:
const                0.121122
Risk-Free Return     0.643587
Risk_Premium_Beta    4.657051
dtype: float64
Information Ratio last year: 0.03091479817864794
1Y Historic Equity Market Stats (filing between): 2017-02-03 08:01:00 and 2018-02-02 08:01:00
const                0.000887
Risk-Free Return     6.226105
Risk_Premium_Beta    1.074529
dtype: float64
With T-Values:
const                0.242531
Risk-Free Return     1.626816
Risk_Premium_Beta    3.151302
dtype: float64
Information Ratio next year:0.061251480476812614
################################################## Next Period#################################################
Report that was published in: 2018q2
      form    fy  fp   fye    period                  adsh  \
2029  10-Q  2018  Q2  0930  20180

Fetching Report that was published in :  20191231
Equity Market Stats for 1Y since Filing on: 2020-01-28 18:03:00
const                0.007676
Risk-Free Return     1.336944
Risk_Premium_Beta    0.934355
dtype: float64
With T-Values:
const                1.392649
Risk-Free Return     0.375327
Risk_Premium_Beta    7.630508
dtype: float64
Information Ratio last year: 0.371692592657782
1Y Historic Equity Market Stats (filing between): 2019-01-29 18:03:00 and 2020-01-28 18:03:00
const                0.006930
Risk-Free Return     3.203805
Risk_Premium_Beta    1.387292
dtype: float64
With T-Values:
const                2.028898
Risk-Free Return     1.587357
Risk_Premium_Beta    6.616001
dtype: float64
Information Ratio next year:0.5983062523638331
################################################## Next Period#################################################


c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Report that was published in: 2020q2
      form    fy   fp   fye    period                  adsh  \
4044   8-K   NaN  NaN  0930  20200430  0000320193-20-000050   
4604  10-Q  2020   Q2  0930  20200331  0000320193-20-000052   
8704   8-K   NaN  NaN  0930  20200430  0001193125-20-139112   

                   accepted  
4044  2020-04-30 16:31:00.0  
4604  2020-04-30 18:03:00.0  
8704  2020-05-11 16:31:00.0  
Fetching Report that was published in :  20200331
Equity Market Stats for 1Y since Filing on: 2020-04-30 18:03:00
const                 0.003036
Risk-Free Return     30.908379
Risk_Premium_Beta     1.062564
dtype: float64
With T-Values:
const                0.579894
Risk-Free Return     2.255712
Risk_Premium_Beta    5.402261
dtype: float64
Information Ratio last year: 0.1492046962300508
1Y Historic Equity Market Stats (filing between): 2019-05-02 18:03:00 and 2020-04-30 18:03:00
const                0.007700
Risk-Free Return     0.865777
Risk_Premium_Beta    0.995146
dtype: float64
W

## Setting Up an Annual Report process

The code needs to be 100% clear as of which date the numbers are and to what period they relate to and when they were published. In any anual report, you might find income numbers for the last quarter, the last 6 months, the last year and the previous year. In the loop I already exclude any periods that are not as of this date, so we have only the relevant dates per year here. The SEC "accepted" date is the date the numbers went public, so our regressions that give future returns since earnings release start there. Historic regressions end there. 

Context:
* Quaterly reports can include unaudited statements on both balance sheet and income figures
* Annual reports will usually include audited statements of both 
* Ideally, any process should be built around the minimum one can expect, i.e. the annual report and then be flexible in terms of using the quaterly reports to get more color

In [296]:
annual_report_long = pd.concat(annual_report_list)
annual_reports_complete_long = pd.pivot_table(annual_report_long, index= "ddate",columns = "tag", values="value")

annual_report_long = pd.concat(annual_report_list)
annual_reports_complete_long = pd.pivot_table(annual_report_long, index= "ddate",columns = "tag", values="value")
point_in_time_numbers_long = annual_report_long[annual_report_long["qtrs"]==0]
annual_numbers_long = annual_report_long[annual_report_long["qtrs"]==4]

point_in_time_numbers_wide = pd.pivot_table(point_in_time_numbers_long, index= "tag",columns = "ddate", values="value")
annual_numbers_wide = pd.pivot_table(annual_numbers_long, index= "tag",columns = "ddate", values="value")

point_in_time_numbers_long = pd.pivot_table(point_in_time_numbers_long, index= "ddate",columns = "tag", values="value")
annual_numbers_long = pd.pivot_table(annual_numbers_long, index= "ddate",columns = "tag", values="value")

market_data_results_wide = pd.concat(market_data_list,axis=1)
market_data_results_long = market_data_results_wide.transpose()
point_in_time_numbers_wide.to_excel("Output/"+str(symbol)+".xlsx")
complete_data_wide = pd.merge(annual_reports_complete_long,market_data_results_long, left_index=True, right_index=True).transpose()
complete_data_long = pd.merge(annual_numbers_long,market_data_results_long, left_index=True, right_index=True)
complete_data_long.index = pd.DatetimeIndex(complete_data_long.index)
complete_data_wide.to_excel("Output/"+str(symbol)+"_complete.xlsx")

### This is relatively clean, but unordered data. Next we need to put this into a structure that we can use for a model.

In [91]:
firms_df[firms_df["SIC"]==sic_sector]

,Firm,SIC,CIK,Ticker
1807,DELL TECHNOLOGIES INC.,3571,1571996,dell
5232,HUAMI CORP,3571,1720446,zepp
6944,"SECTOR 5, INC.",3571,1550737,sfiv
7047,"OMNICELL, INC",3571,926326,omcl
7086,NICE LTD.,3571,1003935,nice
7087,NICE LTD.,3571,1003935,ncsyf
7135,CONCURRENT COMPUTER CORP/DE,3571,749038,ccur
7136,CONCURRENT COMPUTER CORP/DE,3571,749038,ccurd
9901,TERADATA CORP /DE/,3571,816761,tdc
10636,"SUPER MICRO COMPUTER, INC.",3571,1375365,smci


In [104]:
if (period_start in capm_df.index)&(period_end in capm_df.index):
    print("yes")
else:
    print("no")

no


In [112]:
numdict1 = {}
for x,file in enumerate(database): 
    numdict1[file] = num_list[x]

time_long = time.process_time()
database = os.listdir("Database")
for CIK in firms_df[firms_df["SIC"]==sic_sector]["CIK"].unique():

    t = time.process_time()
    #loading database again without zips
    database = os.listdir("Database")
    report_list = list() #this will get all reports
    date_list = list()
    annual_report_list = list() #from this we can make a balance sheet file 
    annual_report_date_list = list() #dates for balance sheet file
    market_data_list = list() #market_data list for regressions etc. 
    share_price_ts = []

    name = str(firms_df[firms_df["CIK"]==CIK]["Firm"].iloc[0])
    print("IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII")
    print("Starting with: "+name)
    for file in database:
        dtypes = {"fy":str,
                  "fye":str,
                  "period":str,
                  "ddate":str,
                  "sic":str,
                 "cik":str}

        sub = pd.read_csv("Database/"+file+"/sub.txt", sep="\t", dtype = dtypes)
        report_type = "10-Q"

        if CIK not in sub["cik"].unique(): #If the firm we are looking for is not in the metadata for that quarter, we dont need to get into numbers
            print(name+" not in "+file)
            continue 

        entity_sub = sub[sub["cik"]==CIK]
        print("Report that was published in: "+file)
        print(entity_sub[["form","fy","fp","fye","period","adsh","accepted"]])
        if "10-Q" not in entity_sub["form"].unique(): #If our firm did not release a 10-Q, we check if we find the 10-K (they do not release both in the quarter after their year end)
            print("No 10-Q in "+file+", fetching 10-K instead")
            report_type = "10-K"
            if "10-K" not in entity_sub["form"].unique(): #If our firm released neither 10-Q nor a 10-K 
                print("No 10-Q or 10-K for "+file+" skipping to next quarter")
                continue
        num = numdict1[file]
        #Get the meta data
        quaterly_report_adsh = entity_sub[entity_sub["form"]==report_type]["adsh"].iloc[0] #fetch report identifier "adsh"
        quaterly_report_period = entity_sub[entity_sub["form"]==report_type]["period"].iloc[0] #fetch report accounting period

        #Get the market data
        first_filing_date = datetime.strptime(entity_sub[entity_sub["form"]==report_type]["accepted"].iloc[0].replace(".0",""), "%Y-%m-%d %H:%M:%S") #fetch report acceptance date

        print("Fetching Report that was published in :  "+str(quaterly_report_period))

        #For the first date for which we have accounting data, download all the data we need for the CAPM 
        if ((date_list==[]) and (type(first_filing_date)==datetime)): #only do this once
            #Get the symbol for stock market API
            #Setting up dictionary
            regression_stats_dict = {"filing_date":None,
                                    "next_year_beta":None,
                                    "next_year_beta_tstat":None,
                                    "next_year_alpha":None,
                                    "next_year_alpha_tstat":None,
                                    "next_year_IR":None,
                                    "next_year_market_return":None,
                                    "next_year_equity_return":None,
                                    "last_year_start_date":None,
                                    "last_year_beta":None, 
                                    "last_year_beta_tstat":None,
                                    "last_year_alpha":None, 
                                    "last_year_alpha_tstat":None,
                                    "last_year_IR": None,
                                    "last_year_market_return":None,
                                    "last_year_equity_return":None,
                                    "last_year_vola":None, 
                                    "last_year_vola_explained_by_market":None}    


            #Getting stock prices and factors 
            start = first_filing_date-pd.Timedelta(52,"W")
            end = datetime.now()

            try:
                share_price_ts = pdr.DataReader(str(firms_df[firms_df["CIK"]==CIK]["Ticker"].iloc[0]), "av-weekly-adjusted", start=start,end=end)["adjusted close"]
            except ValueError:
                try:
                    share_price_ts = pdr.DataReader(str(firms_df[firms_df["CIK"]==CIK]["Ticker"].iloc[1]), "av-weekly-adjusted", start=start,end=end)["adjusted close"]
                except ValueError:
                    print('No Stock Prices available')
                    #Get the accounting data 
                    entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh] #fetch report
                    entity_num_quarterly_period = entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                                       entity_num_quarterly["qtrs"].isin([0,1])] #Only get the data for the last quarter
                    date_list.append(quaterly_report_period)
                    report_list.append(entity_num_quarterly_period)

                    if report_type == "10-K":
                        annual_report_list.append(entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                                        entity_num_quarterly["qtrs"].isin([0,4])])#Get last year data
                        annual_report_date_list.append(quaterly_report_period)
                        print("Also saving this as 10-K report")
                    print("################################################## Next Period#################################################")
                    continue

            return_equity = share_price_ts.pct_change()
            capm_df = pd.concat([return_equity,return_market,return_risk_free], axis=1).dropna()
            capm_df.columns = ["Equity Return", "Market Return", "Risk-Free Return"]
            capm_df["Risk_Premium_Beta"] = capm_df["Market Return"] - capm_df["Risk-Free Return"]         
            capm_df.index = pd.DatetimeIndex(capm_df.index)


            period_start = first_filing_date
            period_end = first_filing_date+pd.Timedelta(52,"W")
            #Next year Stock analysis:
        if len(capm_df[period_start:period_end])>26:
            reg_df = capm_df.loc[period_start:period_end]
            X = reg_df[["Risk-Free Return", "Risk_Premium_Beta"]]
            X = sm.add_constant(X)
            model = sm.OLS(endog = reg_df["Equity Return"], exog = X, missing = "drop")
            results = model.fit()
            information_ratio_capm = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))

            #Printing relevant data
            print("Equity Market Stats for 1Y since Filing on: "+str(first_filing_date))
            print(results.params)
            print("With T-Values:")
            print(results.tvalues)
            print("Information Ratio last year: "+str(information_ratio_capm))

            #Saving Data
            regression_stats_dict["filing_date"] = first_filing_date
            regression_stats_dict["next_year_beta"] = results.params["Risk_Premium_Beta"]
            regression_stats_dict["next_year_beta_tstat"] = results.tvalues["Risk_Premium_Beta"]
            regression_stats_dict["next_year_alpha"] = results.params["const"]
            regression_stats_dict["next_year_alpha_tstat"] = results.tvalues["const"]
            regression_stats_dict["next_year_IR"] = information_ratio_capm
            regression_stats_dict["next_year_market_return"] = (reg_df["Market Return"]+1).iloc[-1]-1
            regression_stats_dict["next_year_equity_return"] = (reg_df["Equity Return"]+1).iloc[-1]-1


            #Make dataframe and store it to list here already if we have not enough data for historic analysis
            if len(capm_df.loc[first_filing_date-pd.Timedelta(52,"w"):first_filing_date])< 26:
                market_data_df = pd.DataFrame.from_dict(regression_stats_dict,orient='index')
                market_data_df.columns = [quaterly_report_period]
                market_data_list.append(market_data_df)

        #Historic Stock Return Analysis if we have 26 weeks of data (1/2 year)
        if len(capm_df.loc[first_filing_date-pd.Timedelta(52,"w"):first_filing_date])>= 26:
            #Setup and run regression
            period_start = first_filing_date-pd.Timedelta(52,"w")
            period_end = first_filing_date
            reg_df = capm_df.loc[period_start:period_end]
            X = reg_df[["Risk-Free Return", "Risk_Premium_Beta"]]
            X = sm.add_constant(X)
            model = sm.OLS(endog = reg_df["Equity Return"], exog = X, missing = "drop")
            results = model.fit()
            information_ratio_capm = ((results.params["const"]+1)**12-1)/(results.resid.std()*np.sqrt(52))

            #Printing relevant data
            print("1Y Historic Equity Market Stats (filing between): "+str(period_start)+" and "+str(period_end))
            print(results.params)
            print("With T-Values:")
            print(results.tvalues)
            print("Information Ratio next year:"+str(information_ratio_capm))

            #saving relevant data
            regression_stats_dict["last_year_start_date"] = reg_df.first_valid_index()
            regression_stats_dict["last_year_beta"] = results.params["Risk_Premium_Beta"]
            regression_stats_dict["last_year_beta_tstat"]= results.tvalues["Risk_Premium_Beta"]
            regression_stats_dict["last_year_alpha"]=results.params["const"]
            regression_stats_dict["last_year_alpha_tstat"]=results.tvalues["const"]
            regression_stats_dict["last_year_IR"] = information_ratio_capm
            regression_stats_dict["last_year_market_return"]=(reg_df["Market Return"]+1).iloc[-1]-1
            regression_stats_dict["last_year_equity_return"]=(reg_df["Equity Return"]+1).iloc[-1]-1
            regression_stats_dict["last_year_vola"]= reg_df["Equity Return"].std()*np.sqrt(len(reg_df))

            #Explaining Vola of Stock with Market: 
            X = reg_df["Equity Return"]
            X = sm.add_constant(X)
            model = sm.OLS(endog = reg_df["Risk_Premium_Beta"], exog = X , missing = "drop")
            results = model.fit()
            regression_stats_dict["last_year_vola_explained_by_market"]=results.params["Equity Return"]

            #Make dataframe and store it to list
            market_data_df = pd.DataFrame.from_dict(regression_stats_dict,orient='index')
            market_data_df.columns = [quaterly_report_period]
            market_data_list.append(market_data_df)

        #Get the accounting data 
        entity_num_quarterly = num[num["adsh"] == quaterly_report_adsh] #fetch report
        entity_num_quarterly_period = entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                           entity_num_quarterly["qtrs"].isin([0,1])] #Only get the data for the last quarter
        date_list.append(quaterly_report_period)
        report_list.append(entity_num_quarterly_period)



        if report_type == "10-K":
            annual_report_list.append(entity_num_quarterly[(entity_num_quarterly["ddate"]==quaterly_report_period)&
                                                            entity_num_quarterly["qtrs"].isin([0,4])])#Get last year data
            annual_report_date_list.append(quaterly_report_period)
            print("Also saving this as 10-K report")

        print("################################################## Next Period#################################################")

    if len(annual_report_list)>1:
        annual_report_long = pd.concat(annual_report_list)
        annual_reports_complete_long = pd.pivot_table(annual_report_long, index= "ddate",columns = "tag", values="value")
    if len(market_data_list)>1:
        market_data_results_wide = pd.concat(market_data_list,axis=1)
        market_data_results_long = market_data_results_wide.transpose()
        complete_data_long = pd.merge(annual_reports_complete_long,market_data_results_long, left_index=True, right_index=True)
        complete_data_long.index = pd.DatetimeIndex(complete_data_long.index)
        complete_data_long.index.name = str(name)+"_"+str(entity_sub["sic"].iloc[0])
        complete_data_long.transpose().to_excel("Output/"+str(name).replace("/","_")+"_complete.xlsx")


    elapsed_time = time.process_time() - t
    print("Processing Time in Minutes: "+str(elapsed_time/60))
    print(str(name)+" Done")
    print("IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII")
    time.sleep(12)
time.process_time()- time_long


IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
Starting with: DELL TECHNOLOGIES INC.
DELL TECHNOLOGIES INC. not in 2009q1
DELL TECHNOLOGIES INC. not in 2009q2
DELL TECHNOLOGIES INC. not in 2009q3
DELL TECHNOLOGIES INC. not in 2009q4
DELL TECHNOLOGIES INC. not in 2010q1
DELL TECHNOLOGIES INC. not in 2010q2
DELL TECHNOLOGIES INC. not in 2010q3
DELL TECHNOLOGIES INC. not in 2010q4
DELL TECHNOLOGIES INC. not in 2011q1
DELL TECHNOLOGIES INC. not in 2011q2
DELL TECHNOLOGIES INC. not in 2011q3
DELL TECHNOLOGIES INC. not in 2011q4
DELL TECHNOLOGIES INC. not in 2012q1
DELL TECHNOLOGIES INC. not in 2012q2
DELL TECHNOLOGIES INC. not in 2012q3
DELL TECHNOLOGIES INC. not in 2012q4
DELL TECHNOLOGIES INC. not in 2013q1
DELL TECHNOLOGIES INC. not in 2013q2
DELL TECHNOLOGIES INC. not in 2013q3
DELL TECHNOLOGIES INC. not in 2013q4
DELL TECHNOLOGIES INC. not in 2014q1
DELL TECHNOLOGIES INC. not in 2014q2
DELL TECHNOLOGIES INC. not in 2014q3

12760  2020-03-27 08:32:00.0  
No 10-Q in 2020q1, fetching 10-K instead
Fetching Report that was published in :  20200131
Equity Market Stats for 1Y since Filing on: 2020-03-27 08:32:00
const               -0.003267
Risk-Free Return    -1.052249
Risk_Premium_Beta    1.314453
dtype: float64
With T-Values:
const               -0.390463
Risk-Free Return    -0.217807
Risk_Premium_Beta    2.705476
dtype: float64
Information Ratio last year: -0.11233296368832933
1Y Historic Equity Market Stats (filing between): 2019-03-29 08:32:00 and 2020-03-27 08:32:00
const                0.000380
Risk-Free Return    -3.880822
Risk_Premium_Beta    1.084504
dtype: float64
With T-Values:
const                0.056704
Risk-Free Return    -1.214770
Risk_Premium_Beta    6.048730
dtype: float64
Information Ratio next year:0.014878438753503403
Also saving this as 10-K report
################################################## Next Period#################################################


c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Report that was published in: 2020q2
       form    fy   fp   fye    period                  adsh  \
486     8-K   NaN  NaN  0131  20200331  0001571996-20-000019   
926     8-K   NaN  NaN  0131  20200331  0001571996-20-000021   
1016    8-K   NaN  NaN  0131  20200331  0001193125-20-102602   
12609   8-K   NaN  NaN  0131  20200531  0001571996-20-000028   
13872  10-Q  2021   Q1  0131  20200430  0001571996-20-000034   

                    accepted  
486    2020-04-03 17:04:00.0  
926    2020-04-08 09:01:00.0  
1016   2020-04-09 16:11:00.0  
12609  2020-05-28 16:29:00.0  
13872  2020-06-08 16:36:00.0  
Fetching Report that was published in :  20200430
Equity Market Stats for 1Y since Filing on: 2020-06-08 16:36:00
const                0.000380
Risk-Free Return    -3.880822
Risk_Premium_Beta    1.084504
dtype: float64
With T-Values:
const                0.056704
Risk-Free Return    -1.214770
Risk_Premium_Beta    6.048730
dtype: float64
Information Ratio last year: 0.014878438753503403
1Y 

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


...

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


################################################## Next Period#################################################
Report that was published in: 2020q3
       form    fy   fp   fye    period                  adsh  \
3049    8-K   NaN  NaN  0930  20200731  0000320193-20-000060   
3567   10-Q  2020   Q3  0930  20200630  0000320193-20-000062   
7052    8-K   NaN  NaN  0930  20200731  0001193125-20-213158   
10735   8-K   NaN  NaN  0930  20200731  0001193125-20-225672   

                    accepted  
3049   2020-07-30 18:55:00.0  
3567   2020-07-30 19:29:00.0  
7052   2020-08-07 17:20:00.0  
10735  2020-08-20 16:31:00.0  
Fetching Report that was published in :  20200630
Equity Market Stats for 1Y since Filing on: 2020-07-30 19:29:00
const                0.007700
Risk-Free Return     0.865777
Risk_Premium_Beta    0.995146
dtype: float64
With T-Values:
const                 2.172379
Risk-Free Return      0.515202
Risk_Premium_Beta    11.917425
dtype: float64
Information Ratio last year: 0.59

Report that was published in: 2019q3
      form    fy  fp   fye    period                  adsh  \
4122  10-Q  2019  Q2  1231  20190630  0001564590-19-030781   

                   accepted  
4122  2019-08-08 16:12:00.0  
Fetching Report that was published in :  20190630
Equity Market Stats for 1Y since Filing on: 2019-08-08 16:12:00
const                 0.002250
Risk-Free Return    -20.958764
Risk_Premium_Beta     0.843808
dtype: float64
With T-Values:
const                0.149035
Risk-Free Return    -1.583852
Risk_Premium_Beta    1.310840
dtype: float64
Information Ratio last year: 0.04155246753371537
1Y Historic Equity Market Stats (filing between): 2018-08-09 16:12:00 and 2019-08-08 16:12:00
const                 0.004620
Risk-Free Return    -20.847380
Risk_Premium_Beta     0.725481
dtype: float64
With T-Values:
const                0.270797
Risk-Free Return    -1.662414
Risk_Premium_Beta    1.107756
dtype: float64
Information Ratio next year:0.07948765727280799
#################

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Report that was published in: 2020q2
       form    fy  fp   fye    period                  adsh  \
10916  10-Q  2020  Q1  1231  20200331  0001564590-20-025550   

                    accepted  
10916  2020-05-14 18:03:00.0  
Fetching Report that was published in :  20200331
Equity Market Stats for 1Y since Filing on: 2020-05-14 18:03:00
const                 0.010477
Risk-Free Return    -14.800636
Risk_Premium_Beta     1.206857
dtype: float64
With T-Values:
const                0.642293
Risk-Free Return    -1.814666
Risk_Premium_Beta    2.419790
dtype: float64
Information Ratio last year: 0.1772091531464115
1Y Historic Equity Market Stats (filing between): 2019-05-16 18:03:00 and 2020-05-14 18:03:00
const                0.012988
Risk-Free Return    -7.364955
Risk_Premium_Beta    1.732529
dtype: float64
With T-Values:
const                0.712520
Risk-Free Return    -0.852969
Risk_Premium_Beta    4.089243
dtype: float64
Information Ratio next year:0.20272326037551502
#################

# Some ideas for supervised learning techniques to predict outperformance 

Idea: 
* For each year we have a number of features (last years B/S and I/S, and various statistical features of stock prices)
* For each year we also have a number of targets (next years performance metrics)
* Let an algorythm train on e.g. 5 years of data for 100 companies to use all accounting data and predict next years performance metrics